In [1]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 7


OpenAI has fine-tuned the gpt-3.5-turbo-0613 and gpt-4-0613 model to:

1. Accept additional arguments through which users can pass in descriptions of functions.

2. If it is relevant, return the name of the function to use, along with a JSON object with the appropriate input parameters.

Lets write a demo function you want to provide it to language model

In [5]:
import json

#function return the same weather

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location":location,
        "temperature":"72",
        "unit": unit,
        "forecast": ["sunny","windy"]
    }
    return json.dumps(weather_info)

openAI has exposed a new parameter called functions through which you can pass a list of functions definitions.

In [6]:
#define a function

functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type":"object",
            "properties": {
                "location":{
                    "type":"string",
                    "description":"The city and state, e.g. San Francisco, CA",
                },
                "unit": {"tyoe":"String", "enum":["celsius","fahrenheit"]},
            },
            "required":["location"]
        },
    }
]

description parameters above are very important as they are directly going to pass to LLM model.

LLM model use these description to determine wheather to use this function and how to call function.

In [8]:
messages = [
    {
        "role":"user",
        "content": "What's the weather like in Boston?"
    }
]

In [17]:
# pip install openai==0.28

In [ ]:
# pip uninstall openai -y

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 1.8 MB/s eta 0:00:00-:--:--
Note: you may need to restart the kernel to use updated packages.


In [32]:
from openai import OpenAI
client = OpenAI()

In [34]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages,
    functions = functions
)

In [ ]:
print(response)

ChatCompletion(id='chatcmpl-BaFSffYssie6sXaTHhkaktPrNtUFm', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1747978941, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=82, total_tokens=102, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [42]:
import json

print(json.dumps(response.model_dump(), indent=2))


{
  "id": "chatcmpl-BaFSffYssie6sXaTHhkaktPrNtUFm",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{\"location\":\"Boston\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1747978941,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 20,
    "prompt_tokens": 82,
    "total_tokens": 102,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }


In [49]:
response_message = response.choices[0].message

In [52]:
print(json.dumps(response_message.model_dump(),indent=2))

{
  "content": null,
  "refusal": null,
  "role": "assistant",
  "annotations": [],
  "audio": null,
  "function_call": {
    "arguments": "{\"location\":\"Boston\",\"unit\":\"celsius\"}",
    "name": "get_current_weather"
  },
  "tool_calls": null
}


In [53]:
print(response_message.content)

None


In [57]:
print(json.dumps(response_message.function_call.model_dump(),indent=2))

{
  "arguments": "{\"location\":\"Boston\",\"unit\":\"celsius\"}",
  "name": "get_current_weather"
}


In [58]:
response_message.function_call.arguments

'{"location":"Boston","unit":"celsius"}'

In [62]:
# json.loads(response_message.function_call.arguments)

In [63]:
args = response_message.function_call.arguments

In [64]:
get_current_weather(args)

'{"location": "{\\"location\\":\\"Boston\\",\\"unit\\":\\"celsius\\"}", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

here we can see function calling in openAI doesn't directly call the function, rather it tells us what function to call

what is the name and what the arguments should be, it is strictly not enforced to return JSON block.

what if the message you pass to llm isn't related at all.

In [67]:
messages = [
    {
        "role":"user",
        "content":"hi!"
    }
]

In [68]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages= messages,
    functions = functions
)

In [69]:
print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaFnmm8ujlPolr699YHOBOmypD5mK",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1747980250,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 76,
    "total_tokens": 86,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


here we can see content is normal but no function call was made, model is determining whether to use function or not.

There are additional parameters that we can pass which will force the model to use the function call.shown below.

default function_call parameter is set to auto.

In [70]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages= messages,
    functions = functions,
    function_call="auto",
)

In [72]:
print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaFrx4uba26elcuafYOaEHaHJqp5c",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1747980509,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 76,
    "total_tokens": 86,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


another mode for function_call we can use is none

In [73]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages= messages,
    functions = functions,
    function_call="none",
)

In [74]:
print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaGDBWTYsDSXdckafjiShTzZTqvGi",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1747981825,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 77,
    "total_tokens": 86,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


what if we set function_call to none but content should call the function then. It will not call the function

In [75]:
messages = [
    {
        "role":"user",
        "content":"What's the weather in Boston!"
    }
]

In [77]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages,
    functions = functions,
    function_call = "none"
)

In [78]:
print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaGG3xwi6m4HmQOxYHBSPPoViPYXA",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Sure! Let me check the weather in Boston for you.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1747982003,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 82,
    "total_tokens": 94,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


forcing to call function

In [79]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages,
    functions = functions,
    function_call = {"name":"get_current_weather"},
)

In [80]:
print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaGHe1iYvn8uZygLZSdEVv6i4DHq5",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{\"location\":\"Boston\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1747982102,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 91,
    "total_tokens": 101,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


what if content is irrelevant but we force to call the function, the model gets confuse, keeps calling for San Franciso, CA

In [82]:
messages = [
    {
        "role":"user",
        "content":"hi!"
    }
]

response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages,
    functions = functions,
    function_call = {"name":"get_current_weather"}
)

print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaGL5QCP29jAQ6vrpshgN1eks3uEb",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{\"location\":\"San Francisco, CA\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1747982315,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 86,
    "total_tokens": 99,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }

if we comment functions and function_calls then prompt token goes down. openai has token limit so constructing messages, be aware of length of messages but also be aware of length of functions been passed.

how to pass function calls and the results of actually doing the function all back into language model.

you want language model to decide which function to call, then run that function, pass it back to language model to get final response

In [85]:
messages.append(response.choices[0].message)

In [90]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

In [ ]:
messages.append(
    {
        "role":"function",
        "name":"get_current_weather",
        "content":observation,
    }
)

In [92]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages,
)

print(json.dumps(response.model_dump(),indent=2))

{
  "id": "chatcmpl-BaGY4v3aczZdFiTZXb6ACyeDS0ti2",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! The current weather in San Francisco, CA is 72\u00b0F with a sunny and windy forecast. How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1747983120,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 84,
    "total_tokens": 112,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
